在TensorFlow 2.0中，渴望执行默认情况下处于打开状态。用户界面直观且灵活（运行一次性操作要容易得多且更快），但这可能会牺牲性能和可部署性。

为了获得最佳性能并使模型可部署到任何地方，请使用 tf.function从程序中制作图表。多亏了AutoGraph，tf.function才可以使用数量惊人的Python代码，但是仍然要提防一些陷阱。

主要要点和建议是：

* 不要依赖于Python的副作用，例如对象突变或列表追加。
* tf.function最适合TensorFlow操作，而不是NumPy操作或Python原语。
* 如有疑问，请使用for x in y成语。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [2]:
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
            error_class))

tf.function您定义的A 就像核心TensorFlow操作：您可以急切地执行它；您可以在图形中使用它；它具有渐变；等等。

In [3]:
# A function is like an op

@tf.function
def add(a, b):
    return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [4]:
# Functions have gradients

@tf.function
def add(a, b):
    return a + b

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: id=38, shape=(), dtype=float32, numpy=1.0>

In [5]:
# You can use functions inside functions

@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=64, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

追踪和多态<br>
Python的动态类型化意味着您可以使用各种参数类型来调用函数，并且Python在每种情况下都会做不同的事情。

另一方面，TensorFlow图需要静态dtypes和形状尺寸。tf.function通过在必要时重新生成功能图来缩小差距。使用的大多数微妙之处都tf.function来自这种追溯行为。

您可以调用带有不同类型参数的函数以查看发生了什么。

In [6]:
# Functions are polymorphic

@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



若要控制跟踪行为，请使用以下技术：

* 创建一个新的tf.function。tf.function保证单独的对象不共享跟踪。
* 使用该get_concrete_function方法获取特定的跟踪
* 指定input_signature在调用tf.function时仅对每个调用图跟踪一次。

In [7]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace
Tracing with Tensor("a:0", dtype=string)
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_91 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_91]


In [8]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


什么时候回溯？<br>
多态tf.function保留了由跟踪生成的具体功能的缓存。缓存键实际上是从函数args和kwargs生成的键的元组。为自tf.Tensor变量生成的键是其形状和类型。为Python原语生成的密钥是其值。对于所有其他Python类型，键都是基于对象的，id()因此对于类的每个实例都独立地跟踪方法。将来，TensorFlow可能会为Python对象添加更复杂的缓存，这些缓存可以安全地转换为张量。

Python或Tensor参数？<br>
通常，Python参数用于控制超参数和图形构造-例如num_layers=10or training=True或nonlinearity='relu'。因此，如果Python参数改变，则必须重新绘制图形。

但是，可能没有使用Python参数来控制图形的构造。在这些情况下，Python值的更改可能会触发不必要的跟踪。以这个训练循环为例，AutoGraph将动态展开该训练循环。尽管有多条迹线，但生成的图实际上是相同的，因此效率较低。

In [9]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


这里的一个简单的解决方法是，在不影响所生成图形的形状的情况下，将您的参数转换为Tensors。

In [10]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


副作用 tf.function<br>
通常，Python副作用（如打印或变异对象）仅在跟踪期间发生。那么，如何可靠地从中引发副作用tf.function呢？

一般的经验法则是仅使用Python副作用来调试跟踪。否则，TensorFlow操作（如tf.Variable.assign，tf.print和）tf.summary是确保每次调用时TensorFlow运行时都将跟踪和执行代码的最佳方法。通常，使用功能样式会产生最佳效果。

In [11]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

f(1)
f(1)
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


如果您想在每次调用时执行Python代码tf.function，tf.py_function则为退出舱门。缺点tf.py_function是它不便携或性能不佳，在分布式（多GPU，TPU）设置中也不能很好地工作。同样，由于tf.py_function必须将其连接到图中，因此它将所有输入/输出转换为张量。

In [12]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect
Python side effect
Python side effect


当心Python状态<br>
许多Python功能（例如生成器和迭代器）都依赖Python运行时来跟踪状态。通常，尽管这些构造在“急切”模式下可以正常工作，但是tf.function由于跟踪行为，内部可能发生许多意外情况。

举一个例子，推进迭代器状态是Python的副作用，因此仅在跟踪期间发生。

In [13]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


如果迭代器是在tf.function中完全生成并使用的，则它应该可以正常工作。但是，可能会跟踪整个迭代器，这可能会导致一个巨大的图。这可能就是您想要的。但是，如果您要在以Python列表表示的大型内存数据集中进行训练，则这可能会生成非常大的图形，并且tf.function不太可能产生加速。

如果要遍历Python数据，最安全的方法是将其包装在tf.data.Dataset中并使用该for x in y惯用法。for当y张量或tf.data.Dataset 时，AutoGraph具有对安全转换循环的特殊支持。

In [14]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
        f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # Some dummy computation.
    return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph


当在数据集的Python包装/ numpy的数据，心的tf.data.Dataset.from_generator对tf.data.Dataset.from_tensors。前者将数据保留在Python中并通过获取数据tf.py_function可能会对性能产生影响，而后者会将数据的副本捆绑为tf.constant()图形中的一个大节点，这可能会影响内存。

通过TFRecordDataset / CsvDataset / etc从文件读取数据。是使用数据的最有效方法，因为TensorFlow本身可以管理数据的异步加载和预取，而无需使用Python。

自动控制依赖项<br>
在一般数据流图上，函数作为编程模型的一个非常吸引人的特性是，函数可以为运行时提供有关代码预期行为的更多信息。

例如，当编写具有多个读取和写入相同变量的代码时，数据流图可能无法自然地编码最初预期的操作顺序。在中tf.function，我们通过引用原始Python代码中语句的执行顺序来解决执行顺序中的歧义。这样，有状态操作在tf.function复制中的顺序将复制Eager模式的语义。

这意味着无需添加手动控件依赖项。tf.function足够聪明，可以添加最少的必需和足够的控件依赖项集，以使代码正确运行。

In [15]:
# Automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

f(1.0, 2.0)  # 10.0

<tf.Tensor: id=418, shape=(), dtype=float32, numpy=10.0>

变数<br>
我们可以使用相同的想法，利用代码的预期执行顺序来简化变量的创建和使用tf.function。不过，有一个非常重要的警告，那就是使用变量可以编写在渴望模式和图形模式下表现不同的代码。

具体来说，当您为每个调用创建一个新的变量时，就会发生这种情况。由于语义的跟踪，tf.function每个调用将重用相同的变量，但是渴望模式将为每个调用创建一个新变量。为防止此错误，tf.function如果它检测到危险的变量创建行为，将引发错误。

In [16]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

with assert_raises(ValueError):
    f(1.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-16-f080b8550f95>:3 f  *
        v = tf.Variable(1.0)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\variables.py:254 _variable_v2_call
        shape=shape)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\eager\def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [17]:
# Non-ambiguous code is ok though

v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [18]:
# You can also create variables inside a tf.function as long as we can prove
# that those variables are created only the first time the function is executed.

class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [19]:
state = []
@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 * x))
        state.append(tf.Variable(state[0] * 3.0))
    return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


使用自动绘图<br>
该签名库完全集成tf.function，它将改写条件和循环依赖于张量在图形动态运行。

tf.cond并tf.while_loop继续使用tf.function，但是以命令式方式编写时，具有控制流的代码通常更易于编写和理解。

In [20]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.246125817 0.116935611 0.765960574 0.237833142 0.781809807]
[0.241273433 0.116405524 0.644574463 0.233448014 0.65374428]
[0.236698195 0.11588259 0.568006158 0.229297653 0.574185193]
[0.232374638 0.115366645 0.513893485 0.225361779 0.518426299]
[0.228280455 0.11485754 0.472973257 0.221622512 0.476484507]
[0.22439602 0.114355117 0.440598518 0.218063965 0.443423778]
[0.220703989 0.113859236 0.414140433 0.214672014 0.416478395]
[0.217188954 0.113369755 0.391982615 0.211434036 0.393959552]
[0.213837177 0.112886541 0.373068154 0.208338708 0.374768704]
[0.210636377 0.112409458 0.356672466 0.20537582 0.358155787]
[0.207575545 0.11193838 0.342279673 0.202536196 0.343588561]
[0.20464474 0.11147318 0.329511046 0.199811488 0.33067733]
[0.201834992 0.111013733 0.318081349 0.197194144 0.319129258]
[0.199138194 0.110559925 0.307771027 0.194677293 0.308719367]
[0.196546957 0.110111646 0.298407942 0.192254648 0.299271584]
[0.194054559 0.109668776 0.289854974 0.189920455 0.290645868]
[0.191654861 0.10

<tf.Tensor: id=675, shape=(5,), dtype=float32, numpy=
array([0.18495807, 0.10794932, 0.26180476, 0.18136925, 0.26238617],
      dtype=float32)>

In [21]:
# If you're curious you can inspect the code autograph generates.
# It feels like reading assembly language, though.

def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x):
      ag__.converted_call(tf.print, f_scope.callopts, (x,), None, f_scope)
      x = ag__.converted_call(tf.tanh, f_scope.callopts, (x,), None, f_scope)
      return x,

    def loop_test(x):
      return ag__.converted_call(tf.reduce_sum, f_scope.callopts, (x,), None, f_scope) > 1
    x, = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x,), ('x',), ())
    do_return = True
    retval_ = f_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



AutoGraph：条件<br>
AutoGraph会将if语句转换为等效tf.cond调用。

如果条件为张量，则进行此替换。否则，条件将在跟踪期间执行。

In [22]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print("{}({}) uses tf.cond.".format(
            f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) executes normally.".format(
            f.__name__, ', '.join(map(str, args))))

In [23]:
@tf.function
def hyperparam_cond(x, training=True):
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return x

@tf.function
def maybe_tensor_cond(x):
    if x < 0:
        x = -x
    return x

test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))
test_tf_cond(maybe_tensor_cond, tf.constant(-1))
test_tf_cond(maybe_tensor_cond, -1)

hyperparam_cond(tf.Tensor([1.], shape=(1,), dtype=float32)) executes normally.
maybe_tensor_cond(tf.Tensor(-1, shape=(), dtype=int32)) uses tf.cond.
maybe_tensor_cond(-1) executes normally.


tf.cond有许多微妙之处。-它通过跟踪条件的两端，然后根据条件在运行时选择适当的分支来工作。跟踪两侧可能会导致Python代码意外执行-它要求如果一个分支创建了在下游使用的张量，则另一个分支也必须创建该张量。

In [24]:
@tf.function
def f():
    x = tf.constant(0)
    if tf.constant(True):
        x = x + 1
        print("Tracing `then` branch")
    else:
        x = x - 1
        print("Tracing `else` branch")
    return x

f()

Tracing `then` branch
Tracing `else` branch


<tf.Tensor: id=747, shape=(), dtype=int32, numpy=1>

In [25]:
@tf.function
def f():
    if tf.constant(True):
        x = tf.ones([3, 3])
    return x

# Throws an error because both branches need to define `x`.
with assert_raises(ValueError):
    f()

Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-25-810946e9b87f>:3 f  *
        if tf.constant(True):
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:893 if_stmt
        basic_symbol_names, composite_symbol_names)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:931 tf_if_stmt
        error_checking_orelse)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\util\deprecation.py:507 new_func
        return func(*args, **kwargs)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\control_flow_ops.py:1174 cond
        return cond_v2.cond_v2(pred, true_fn, false_fn, name)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\cond_v2.py:91 cond_v2
        op_return_value=pred)
    c:\users\sha\anaconda3\

AutoGraph和循环<br>
AutoGraph具有一些用于转换循环的简单规则。

* for：如果iterable是张量，则转换
* while：如果while条件取决于张量，则进行转换

如果转换了一个循环，它将动态转换为tf.while_loop，或者在特殊情况下（a for x in tf.data.Dataset）转换为tf.data.Dataset.reduce。

如果循环未转换，它将被静态展开

In [26]:
def test_dynamically_unrolled(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'while' for node in g.as_graph_def().node):
        print("{}({}) uses tf.while_loop.".format(
            f.__name__, ', '.join(map(str, args))))
    elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
        print("{}({}) uses tf.data.Dataset.reduce.".format(
            f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) gets unrolled.".format(
            f.__name__, ', '.join(map(str, args))))

In [27]:
@tf.function
def for_in_range():
    x = 0
    for i in range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [28]:
@tf.function
def for_in_tfrange():
    x = tf.constant(0, dtype=tf.int32)
    for i in tf.range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop.


In [29]:
@tf.function
def for_in_tfdataset():
    x = tf.constant(0, dtype=tf.int64)
    for i in tf.data.Dataset.range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce.


In [31]:
@tf.function
def while_py_cond():
    x = 5
    while x > 0:
        x -= 1
    return x

test_dynamically_unrolled(while_py_cond)

while_py_cond() gets unrolled.


In [32]:
@tf.function
def while_tf_cond():
    x = tf.constant(5)
    while x > 0:
        x -= 1
    return x


test_dynamically_unrolled(while_tf_cond)

while_tf_cond() uses tf.while_loop.


如果您有一个break或一个return取决于张量的早期子句，则顶级条件或可迭代值也应为张量。

比较以下示例：

In [33]:
@tf.function
def while_py_true_py_break(x):
    while True:  # py true
        if x == 0: # py break
            break
        x -= 1
    return x

test_dynamically_unrolled(while_py_true_py_break, 5)

while_py_true_py_break(5) gets unrolled.


In [34]:
@tf.function
def buggy_while_py_true_tf_break(x):
    while True:   # py true
        if tf.equal(x, 0): # tf break
            break
        x -= 1
    return x

with assert_raises(TypeError):
    test_dynamically_unrolled(buggy_while_py_true_tf_break, 5)

Caught expected exception 
  <class 'TypeError'>: in converted code:

    <ipython-input-34-148e37f3ea71>:3 buggy_while_py_true_tf_break  *
        while True:   # py true
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:730 while_stmt
        return _py_while_stmt(test, body, get_state, set_state, init_vars, opts)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:845 _py_while_stmt
        while test(*loop_vars):
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\framework\ops.py:765 __bool__
        self._disallow_bool_casting()
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\framework\ops.py:531 _disallow_bool_casting
        "using a `tf.Tensor` as a Python `bool`")
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\framework\ops.py:518 _

In [35]:
@tf.function
def while_tf_true_tf_break(x):
    while tf.constant(True): # tf true
        if x == 0:  # py break
            break
        x -= 1
    return x

test_dynamically_unrolled(while_tf_true_tf_break, 5)

while_tf_true_tf_break(5) uses tf.while_loop.


In [36]:
@tf.function
def buggy_py_for_tf_break():
    x = 0
    for i in range(5):  # py for
        if tf.equal(i, 3): # tf break
            break
        x += i
    return x

with assert_raises(TypeError):
    test_dynamically_unrolled(buggy_py_for_tf_break)

Caught expected exception 
  <class 'TypeError'>: in converted code:

    <ipython-input-36-b5619b3e6d52>:4 buggy_py_for_tf_break  *
        for i in range(5):  # py for
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:339 for_stmt
        return _py_for_stmt(iter_, extra_test, body, get_state, set_state, init_vars)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:348 _py_for_stmt
        if extra_test is not None and not extra_test(*state):
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\framework\ops.py:765 __bool__
        self._disallow_bool_casting()
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\framework\ops.py:531 _disallow_bool_casting
        "using a `tf.Tensor` as a Python `bool`")
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\

In [37]:
@tf.function
def tf_for_py_break():
    x = 0
    for i in tf.range(5): # tf for
        if i == 3:  # py break
            break
        x += i
    return x

test_dynamically_unrolled(tf_for_py_break)

tf_for_py_break() uses tf.while_loop.


为了累积来自动态展开循环的结果，您需要使用tf.TensorArray。

In [38]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
    return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
  # [batch, time, features] -> [time, batch, features]
    input_data = tf.transpose(input_data, [1, 0, 2])
    max_seq_len = input_data.shape[0]

    states = tf.TensorArray(tf.float32, size=max_seq_len)
    state = initial_state
    for i in tf.range(max_seq_len):
        state = rnn_step(input_data[i], state)
        states = states.write(i, state)
    return tf.transpose(states.stack(), [1, 0, 2])
  
dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

<tf.Tensor: id=1254, shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.35653508, 0.01149547, 0.9093461 , 0.5242202 ],
        [0.39132583, 0.92087126, 1.0783565 , 0.8690239 ],
        [0.78924966, 1.7890056 , 1.484895  , 1.1597422 ]],

       [[0.55073607, 0.6544552 , 0.74741924, 0.5474757 ],
        [0.6120492 , 1.6240565 , 1.2154746 , 0.8164532 ],
        [0.6337079 , 1.8681042 , 1.2629682 , 1.5568563 ]]], dtype=float32)>

与一样tf.cond，tf.while_loop还带有一些微妙之处。-由于循环可执行0次，因此必须在循环上方初始化while_loop下游使用的所有张量-所有循环变量的shape / dtype必须与每次迭代保持一致

In [39]:
@tf.function
def buggy_loop_var_uninitialized():
    for i in tf.range(3):
        x = i
    return x

with assert_raises(ValueError):
    buggy_loop_var_uninitialized()

Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-39-fb9c665fb220>:3 buggy_loop_var_uninitialized  *
        for i in tf.range(3):
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:315 for_stmt
        composite_symbol_names)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:419 _tf_range_for_stmt
        _disallow_undefs_into_loop(*init_vars)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:97 _disallow_undefs_into_loop
        ' before the loop: {}'.format(tuple(s.symbol_name for s in undefined)))

    ValueError: TensorFlow requires that the following symbols must be defined before the loop: ('x',)



In [40]:
@tf.function
def f():
    x = tf.constant(0)
    for i in tf.range(3):
        x = i
    return x

f()

<tf.Tensor: id=1309, shape=(), dtype=int32, numpy=2>

In [41]:
@tf.function
def buggy_loop_type_changes():
    x = tf.constant(0, dtype=tf.float32)
    for i in tf.range(3): # Yields tensors of type tf.int32...
        x = i
    return x

with assert_raises(tf.errors.InvalidArgumentError):
    buggy_loop_type_changes()

Got unexpected exception 
  <class 'TypeError'>: in converted code:

    <ipython-input-41-f464a413782d>:4 buggy_loop_type_changes  *
        for i in tf.range(3): # Yields tensors of type tf.int32...
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:315 for_stmt
        composite_symbol_names)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:769 _tf_while_stmt
        aug_init_vars, **opts)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\while_v2.py:198 while_loop
        add_control_dependenc

In [42]:
@tf.function
def buggy_concat():
    x = tf.ones([0, 10])
    for i in tf.range(5):
        x = tf.concat([x, tf.ones([1, 10])], axis=0)
    return x

with assert_raises(ValueError):
    buggy_concat()

Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-42-df6f2beb378a>:4 buggy_concat  *
        for i in tf.range(5):
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:315 for_stmt
        composite_symbol_names)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:769 _tf_while_stmt
        aug_init_vars, **opts)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\while_v2.py:198 while_loop
        add_control_dependencies=add_control_dependencies)
    c:\users\sha

In [43]:
@tf.function
def concat_with_padding():
    x = tf.zeros([5, 10])
    for i in tf.range(5):
        x = tf.concat([x[:i], tf.ones([1, 10]), tf.zeros([4-i, 10])], axis=0)
        x.set_shape([5, 10])
    return x

concat_with_padding()

<tf.Tensor: id=1432, shape=(5, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>